In [8]:
#! pip install mlflow boto3 awscli

In [ ]:
import mlflow
# step 1 : Set up the mlflow tracking server 
mlflow.set_tracking_uri("http://ec2-51-21-223-34.eu-north-1.compute.amazonaws.com:5000/")

In [10]:
#Set or create an experiment
mlflow.set_experiment("Exp 3 - BOW Trigram max_features")

<Experiment: artifact_location='s3://reddit-reccomender-bucket/9', creation_time=1763312489517, experiment_id='9', last_update_time=1763312489517, lifecycle_stage='active', name='Exp 3 - BOW Trigram max_features', tags={}>

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import mlflow
import mlflow.sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os

In [12]:
df = pd.read_csv('reddit_preprocessing.csv').dropna(subset=['clean_comment'])
df.shape

(36662, 2)

In [ ]:
#Step 0:train test split
X_train,X_test,y_train,y_test = train_test_split(
        df['clean_comment'],df['category'],
        test_size=0.2,
        random_state=42,
        stratify=df['category']
    )

#step1:Function to run the experiment
def run_experiment_BOW_max_features(max_features,X_train_raw,X_test_raw):
    ngram_range = (1,3)

    #step 2 :Vectorization using BOW with varying max_features
    vectorizer =CountVectorizer(ngram_range=ngram_range,max_features=max_features)

    X_train_vec = vectorizer.fit_transform(X_train_raw)
    X_test_vec= vectorizer.transform(X_test_raw)

    # Step 4: Define and train a Random Forest model
    with mlflow.start_run() as run:
        # Set tags for the experiment and run
        mlflow.set_tag("mlflow.runName", f"BOW_Trigrams_max_features_{max_features}")
        mlflow.set_tag("experiment_type", "feature_engineering")
        mlflow.set_tag("model_type", "RandomForestClassifier")

        # Add a description
        mlflow.set_tag("description", f"RandomForest with BOW Trigrams, max_features={max_features}")

        # Log vectorizer parameters
        mlflow.log_param("vectorizer_type", "BOW")
        mlflow.log_param("ngram_range", ngram_range)
        mlflow.log_param("vectorizer_max_features", max_features)

        # Log Random Forest parameters
        n_estimators = 200
        max_depth = 15

        mlflow.log_param("n_estimators", n_estimators)
        mlflow.log_param("max_depth", max_depth)

        # Initialize and train the model
        model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
        model.fit(X_train_vec, y_train)

         # Step 5: Make predictions and log metrics
        y_pred = model.predict(X_test_vec)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log confusion matrix
        conf_matrix = confusion_matrix(y_test,y_pred)
        plt.figure(figsize=(8,6))
        sns.heatmap(conf_matrix,annot=True,fmt='d',cmap="Blues")
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title(f"Confusion Matrix: BOW Trigrams, max_features={max_features}")
        filename = f"confusion_matrix/conf_matrix_{max_features}_{ngram_range}.png"
        plt.savefig(filename)
        mlflow.log_artifact(filename)
        plt.close()

        # Log the model
        mlflow.sklearn.log_model(model, f"random_forest_model_BOW_trigrams_{max_features}")

# Step 6: Test various max_features values
max_features_values = [1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000]

for max_features in max_features_values:
    run_experiment_BOW_max_features(max_features,X_train,X_test)



2025/11/16 23:13:12 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/16 23:13:42 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run BOW_Trigrams_max_features_1000 at: http://ec2-56-228-82-122.eu-north-1.compute.amazonaws.com:5000/#/experiments/9/runs/c82f399735704d7481d7b9bf36150399
🧪 View experiment at: http://ec2-56-228-82-122.eu-north-1.compute.amazonaws.com:5000/#/experiments/9


2025/11/16 23:14:23 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/16 23:14:40 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run BOW_Trigrams_max_features_2000 at: http://ec2-56-228-82-122.eu-north-1.compute.amazonaws.com:5000/#/experiments/9/runs/b6bf3f350c5a4d8e8275412ca7d243af
🧪 View experiment at: http://ec2-56-228-82-122.eu-north-1.compute.amazonaws.com:5000/#/experiments/9


2025/11/16 23:15:17 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/16 23:15:37 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


...

🏃 View run BOW_Trigrams_max_features_8000 at: http://ec2-56-228-82-122.eu-north-1.compute.amazonaws.com:5000/#/experiments/9/runs/8cdd5e58db424bbba9df71f41dae0a67
🧪 View experiment at: http://ec2-56-228-82-122.eu-north-1.compute.amazonaws.com:5000/#/experiments/9


2025/11/16 23:20:25 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/16 23:20:41 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run BOW_Trigrams_max_features_9000 at: http://ec2-56-228-82-122.eu-north-1.compute.amazonaws.com:5000/#/experiments/9/runs/74e29ec7af97485b8d000489b79f2589
🧪 View experiment at: http://ec2-56-228-82-122.eu-north-1.compute.amazonaws.com:5000/#/experiments/9


2025/11/16 23:21:11 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/16 23:21:25 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run BOW_Trigrams_max_features_10000 at: http://ec2-56-228-82-122.eu-north-1.compute.amazonaws.com:5000/#/experiments/9/runs/8be4cfc74f914884bb3734c57d601c7c
🧪 View experiment at: http://ec2-56-228-82-122.eu-north-1.compute.amazonaws.com:5000/#/experiments/9
